In [1]:
from numpy import argmax
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model


from numpy import array
from pickle import load
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
import json
# load and show an image with Pillow
from matplotlib import image
from matplotlib import pyplot
from PIL import Image
from pickle import dump
from pickle import load

In [3]:
num_to_test = 50
num_to_train = 500

In [4]:

# extract features from each photo in the directory
def extract_features(annotation):
	# load the model
	model = VGG16()
	# re-structure the model
	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	# summarize
	print(model.summary())
	# extract features from each photo
	features = dict()

	for i in range(num_to_test):
		# load an image from file
		filename = ''

		filename = 'Dataset/val/'+annotation['images'][i]['file_name']
		image = load_img(filename, target_size=(224, 224))
		# convert the image pixels to a numpy array
		image = img_to_array(image)
		# reshape data for the model
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		# prepare the image for the VGG model
		image = preprocess_input(image)
		# get features
		feature = model.predict(image, verbose=0)
		# store feature
		features[i] = feature
		# print('>%s' % name)
	return features

In [5]:
import string

def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)

In [6]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

In [7]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(str(key) + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [8]:
# load photo features
def load_photo_features(filename, num):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in range(num)}
	return features

In [9]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
# load clean descriptions into memory
def load_clean_descriptions(filename, num):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = int(tokens[0]), tokens[1:]
		# skip images not in the set
		if image_id in range(num):
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

In [10]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer


In [11]:
# calculate the length of the description with the most words
def max_lengths(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)
 
# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, desc_list, photo, vocab_size):
	X1, X2, y = list(), list(), list()
	#print(desc_list)
	# walk through each description for the image
	for desc in desc_list:
		# print(desc)
		# encode the sequence
		seq = tokenizer.texts_to_sequences([desc])[0]
		# split one sequence into multiple X,y pairs
		for i in range(1, len(seq)):
			# split into input and output pair
			in_seq, out_seq = seq[:i], seq[i]
			# pad input sequence
			in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
			# encode output sequence
			out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
			# store
			X1.append(photo)
			X2.append(in_seq)
			y.append(out_seq)
	return array(X1), array(X2), array(y)

In [12]:
test_annotation = json.load(open("Dataset/annotations/val.json"))
print(test_annotation['info'])
test_descriptions = dict()
for i in range(num_to_test):
    image = Image.open('Dataset/val/'+test_annotation ['images'][i]['file_name'])
    image = image.convert(mode='L')
    image = image.resize((640, 480))
    print(image.size)
    test_descriptions[i] = []
    for j in range(5):
        print('Caption #',j,': ',test_annotation['annotations'][i*5+j]['caption'])
        test_descriptions[i].append(test_annotation['annotations'][i*5+j]['caption'])
test_features = extract_features(test_annotation)
print('Extracted Features: %d' % len(test_features))
# save to file
dump(test_features, open('test_features.pkl', 'wb'))

{'description': 'This dataset contains crowdsourced captions of images from VizWiz datasets. This file contains the val partition.', 'license': {'url': 'https://creativecommons.org/licenses/by/4.0/', 'name': 'Attribution 4.0 International (CC BY 4.0)'}, 'url': 'https://vizwiz.org', 'version': 'VizWiz-Captions 1.0', 'year': 2019, 'contributor': 'VizWiz-Captions Consortium', 'date_created': '2019-12-23'}
(640, 480)
Caption # 0 :  A computer screen shows a repair prompt on the screen.
Caption # 1 :  a computer screen with a repair automatically pop up
Caption # 2 :  partial computer screen showing the need of repairs
Caption # 3 :  Part of a computer monitor showing a computer repair message.
Caption # 4 :  The top of a laptop with a blue background and dark blue text.
(640, 480)
Caption # 0 :  A person is holding a bottle that has medicine for the night time.
Caption # 1 :  A bottle of medication has a white twist top.
Caption # 2 :  night time medication bottle being held by someone
Cap

In [13]:
clean_descriptions(test_descriptions)
# save descriptions
save_descriptions(test_descriptions, 'test_descriptions.txt')
test_descriptions = load_clean_descriptions('test_descriptions.txt', num_to_test)
test_features = load_photo_features('test_features.pkl', num_to_test)


In [14]:
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', num_to_train)
print('Descriptions: train=%d' % len(train_descriptions))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_lengths(train_descriptions)
print('Description Length: %d' % max_length)

Descriptions: train=500
Vocabulary Size: 2657
Description Length: 52


In [15]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [16]:
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

In [21]:
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    # step over the whole set
    for key, desc_list in descriptions.items():
        # generate description
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        print(key,yhat)
        # store actual and predicted
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat.split())
	# calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))


In [22]:
# load the model
filename = 'Model\model_19.h5'
model = load_model(filename)
# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

0 startseq computer monitor is on utility monitor endseq
1 startseq white bottle of medicine sitting on top of white table endseq
2 startseq card with graphics on the back of the nutritional chopper endseq
3 startseq us open red and white bill on top of it endseq
4 startseq computer monitor you avatar mix endseq
5 startseq computer and white menu you text text on the wall endseq
6 startseq quality issues are too severe to recognize visual content endseq
7 startseq quality issues are too severe to recognize visual content endseq
8 startseq quality issues are too severe to recognize visual content endseq
9 startseq quality issues are too severe to recognize visual content endseq
10 startseq house with tree and tree on it endseq
11 startseq purple shirt worn up up on bed endseq
12 startseq an type of paper describes this instructions endseq
13 startseq quality issues are too severe to recognize visual content endseq
14 startseq quality issues are too severe to recognize visual content end